# (4) Identify Location of F Points by Tide 

**Author:** Bryony Freer

**Date Published:** 31 May 2023

Per RGT: 

1. Read in the TideInfo csv file created in the **Plot Tide Distribution for XYT** notebook.
2. For each GT, for each cycle, read in the CSV file of elevation anomalies/lat/long created in **ATL06_PlotRepeatTracks**
3. Apply Low-Pass Butterworth Filter 
4. Calculate 2nd derivative of this 
5. Use peaks of 2nd derivative to identify the location of F (for some tracks need to specify latitude limits to make sure we pick the right one, so visual inspection of each result is necessary to sense-check)  
6. Extracts lat/long of the position of the 2nd derivative peak from the original elevation anomalies file. 
6. Save figures plotting anomalies, low pass filter and 2nd derivative 
7. Exports CSV of **Track{rgt}_{gt}\_F\_locations_auto.csv'**, which includes: 

* rgt	
* cyc	
* rgt_cyc - unique identifier combining rgt and cycle number - e.g. 1016_06
* tide_h - tide amplitude (m)
* tide_grad - tide gradient (cm/min)
* pc_above - percentage of total tide range above this tide 
* pc_below - percentage of total tide range below this tide 
* tide_x - x coord where tide range for region was calculated 
* tide_y - y coord where tide range for region was calculated 
* F_lat - latitude of automatically picked F point 
* F_lon - longitude of automatically picked F point 

In [ ]:
#Package imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.interpolate import splev, splrep
from scipy import signal
from math import sin, cos, sqrt, atan2, radians
import os

from ATL06_functions_published import create_color_dict, distance

import ipywidgets as widgets
# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
# Set directory locations
region = 'FRIS' 
root = '' # Path to directory containing data and outputs
results_dir = '' # Directory containing 'Track{rgt}_TideInfo.csv'
outdir = results_dir

# Set consistent colour dictionary for different cycles (currently up to Cycle 17)
color_dict = create_color_dict()

### Identify F Points by Tide per RGT (or multiple RGTs)

In [ ]:
plot_fig = True #Set to True to plot figures in the notebook per cycle per track 
plot_mig_dist = True #Set to True to plot figure of migration distance vs tide height per track

save_fig = False #Set to True to export figures 
export_csv = False #Set to True to export csv with F locations

#Set figure parameters 
figsize=[9,5]
legend=False # set to True if want to plot legend 

#Optional - set latitude mask 
lat_mask_flag = False #True #True #set to False if want to use all the track
lat_limit = None

In [ ]:
# Set RGTs 

#Option 1: Read in CSV of used RGTs 
#rgts_df = pd.read_csv('used-rgts-and-lowest-tide-info.csv', index_col='rgt')
#rgts = [str(x).zfill(4) for x in rgts_df.index.values]

#Option 2: manually specify RGTs 
rgts = ['0559']

In [ ]:
for rgt in rgts:
    print('RGT: ', rgt)
    #Create output directories 
    plot_outdir=outdir + f'Track{rgt}/F_id_plots/'
    csv_outdir= root + region + '/F_Locations/' 
    if not os.path.isdir(plot_outdir):
        os.makedirs(plot_outdir)
    if not os.path.isdir(csv_outdir):
        os.makedirs(csv_outdir)
        
    #Import csv with tide info per cycle and extract cycle info (including lowest)
    df=pd.read_csv(f'{results_dir}Track{rgt}/Track{rgt}_TideInfo.csv', sep=',', index_col=0)
    cycles = df['cyc'].values
    lowest = df.query(f'tide_h=={df["tide_h"].values.min()}')['cyc']
    lowest = str(lowest.values[0]).zfill(2)
    print('Cycles: ', cycles, '\nlowest tide cycle: ', lowest)
    
    for rpt in [1,2,3]:
        for lr in ['L', 'R']:
            print(rgt, rpt, lr)
            F = []
            F_lats = [] 
            F_lons = []
            cycs_to_drop = [] 
            df['rpt']=f'{rpt}{lr}'
            for c in cycles:
                #print('Cycle ', c)
                cyc = str(c).zfill(2)
                if cyc == lowest:
                    cycs_to_drop.append(lowest)
                    continue
                
                #Option: discard certain cycles if needed
#                 if cyc == '06':
#                     cycs_to_drop.append(cyc)
#                     continue

                try:
                    anom_df = pd.read_csv(f'{results_dir}Track{rgt}/Track{rgt}_GT{rpt}{lr}_cyc{cyc}_lowest{lowest}_v1.csv', sep=',', index_col=0)
                except FileNotFoundError as E:
                    #if the file is not found, may be as this GT didn't have data for one cycle that another did. So fill values with NaN.
                    print(E)
                    F.append(np.nan)
                    F_lats.append(np.nan)
                    F_lons.append(np.nan)
                    continue
                        
                if lat_mask_flag == True:
                    lat_mask = anom_df['lat_corr']>lat_limit
                    anom = anom_df['anom'][lat_mask].values
                    lats = anom_df['lat_corr'][lat_mask].values
                else:
                    anom = anom_df['anom'].values
                    lats = anom_df['lat_corr'].values
                    
                if plot_fig == True:
                    fig,ax = plt.subplots(figsize=figsize)

                #Apply butterworth low-pass filter (order 5)
                w = 0.016 #normalised cut off frequency 
                b,a = scipy.signal.butter(5,w)
                anom_filt = signal.filtfilt(b,a,anom)
                if plot_fig == True:
                    ln1 = ax.scatter(lats, anom, label='Elevation Anomaly',color=color_dict[int(cyc)],s=2)
                    ln2 = ax.plot(lats, anom_filt, label='Low Pass Filter',color='k')
                    ax.set_ylabel('Elevation Anomaly (m)',size=14)
                    ax.set_xlabel('Latitude', size=14)

                    #Add first and second derivative to plot 
                    ax2 = ax.twinx()
                
                grad = np.gradient(anom_filt) #calculate 1st derivative
                grad2 = np.gradient(grad) #calculate 2nd derivative 
                if plot_fig == True:
                    ln4 = ax2.plot(lats,grad2, color='green', label='2nd derivative')
                    ax2.set_ylabel('2nd Derivative',size=14)

                #find the x value at the maximum value of 2nd derivative (limiting to where y<0.25 to avoid picking further seaward point)
                lats_filt = lats[(anom_filt<0.25)]
                grad2_filt=grad2[(anom_filt<0.25)]
                try:
                    max2d_x = lats_filt[grad2_filt.argmax()]
                except ValueError as E:
                    print(rgt, rpt, lr, c, E, ': max2d_x set to nan.')
                    max2d_x = np.nan
               
                '''
                CODE TO INSERT INTO MAIN BLOCK WHERE APPROPRIATE TO MANUALLY ADJUST CHOICE OF PEAK (by setting latitude limits for particular cycles where needed.) )
                
                # Example to adjust using latitude range 
                if cyc in ['06']:
                    min_lat = -80.96
                    max_lat = -80.94
                    lats_filt = lats[(anom_filt<0.25)&(lats>min_lat)&(lats<max_lat)]
                    grad2_filt=grad2[(anom_filt<0.25)&(lats>min_lat)&(lats<max_lat)]
                    max2d_x = lats_filt[grad2_filt.argmax()]
                    print(f'Cycle {cyc}: Applied latitude filter to select correct peak of 2nd Derivative at: {max2d_x}')  
                    
                '''

                F.append(max2d_x) #Append to list of F values to  be exported
                if plot_fig == True:
                    print(max2d_x)
                    #Plot location of 2nd derivative peak as vertical line
                    ax2.axvline(max2d_x, linestyle='dashed',color=color_dict[int(cyc)])  


                #find the lat-long pair in original anom_df dataset with lat closest to max2d_x
                ### a) deal with nan values 
                if max2d_x == np.nan:
                    F_lats.append(np.nan)
                    F_lons.append(np.nan)
                else: 
                    ### b) find index of anom_df with latitude closest to max2d_x
                    idx = anom_df['lat_corr'].sub(max2d_x).abs().idxmin()
                    ### c) extract lat-long pair at that index
                    lat = anom_df['lat_corr'].iloc[idx]
                    lon = anom_df['lon_corr'].iloc[idx]
                    F_lats.append(lat)
                    F_lons.append(lon)

                if plot_fig == True:
                    if legend==True:#Add all labels to single legend
                        lns = ln2+ln4 #ln1+
                        labs = [l.get_label() for l in lns]
                        ax.legend(lns, labs, loc='lower left',bbox_to_anchor=(0,-0.4))

                    #Add horizontal line at y=0 (i.e. lowest tide cycle baseline)
                    ax.axhline(y=0,color=color_dict[int(lowest)])

                    plt.title(f'RGT {rgt} GT{rpt}{lr}, Cycle {cyc}', weight='bold', size=16)
                    plt.tight_layout()
                if save_fig == True:
                    figname = plot_outdir + f'Track{rgt}_GT{rpt}{lr}_cyc{cyc}_F_id_plot'
                    if os.path.isfile(figname):
                        os.remove(figname)
                    plt.savefig(figname + '.png', dpi=300) #save as PNG image (raster)
                    plt.savefig(figname + '.eps', dpi=300) #save as EPS file (vector)
            
            try:
                for cycs in cycs_to_drop:
                    df.drop(df.index[df['cyc'] == int(cycs)], inplace=True) #remove the row of the lowest tide cycle (as anomalies calculated against this, so are meaningless). Also drops any cycles we have discarded.   
                    print('dropped cycle ', cycs)
                df.set_index('cyc',inplace=True) #set the cycle number column as the index
            except KeyError:
                print('Index already changed to cycle. Key Error, but fine to continue.')
            
            df['F'] = F
            df['F_lat'] = F_lats
            df['F_lon'] = F_lons

            #Calculate distances between F points: 
            ### a) get cycle with lowest F tide (to calculate the distances against) 
            
            ### b) get lat and lon of F point at min tide 
            lat1 = df.loc[Fmin_tide_cyc]['F_lat']
            lon1 = df.loc[Fmin_tide_cyc]['F_lon']

            ### c) calculate distance (km) between F point of each cycle and the F point at lowest tide 
            distances= []
            for i in range(0,len(df.index)):
                lat2 = df.iloc[i]['F_lat']
                lon2 = df.iloc[i]['F_lon']
                dist = distance(lat1,lon1,lat2,lon2)
                distances.append(dist)
            df['cyc_Fmin_tide'] = Fmin_tide_cyc
            df['dist_F_to_Fmin_tide'] = distances
            df['lowest_overall_tide'] = lowest

            #Export to CSV (to be read into QGIS)
            if export_csv == True:
                filename = f'{csv_outdir}/Track{rgt}_GT{rpt}{lr}_F_locations_auto.csv'
                if os.path.isfile(filename):
                    os.remove(filename)
                df.to_csv(filename)
                
            if plot_mig_dist == True: 
                plt.figure(figsize=[6,3]) 
                for i in df.index:

                    if df.loc[i]['tide_grad']>0:
                        plt.scatter(df.loc[i]['tide_h'], df.loc[i]['dist_F_to_Fmin_tide'], color=color_dict[int(i)], label=f'Cycle {i}', marker='^')
                    else:
                        plt.scatter(df.loc[i]['tide_h'], df.loc[i]['dist_F_to_Fmin_tide'], color=color_dict[int(i)], label=f'Cycle {i}', marker='v')

                plt.ylabel('Migration Distance (km)')
                plt.xlabel('Tide Height (m)')
                plt.title(f'Migration of Flexure Limit From Lowest Sampled Tide\n RGT {rgt} GT{rpt}{lr}')
                plt.tight_layout()
                if save_fig==True:
                    fig_name = plot_outdir + f'Track{rgt}_GT{rpt}{lr}_mig_dist'
                    if os.path.isfile(fig_name):
                        os.remove(fig_name)
                    plt.savefig(fig_name + '.png', dpi=300) #save as PNG image (raster)
                    plt.savefig(fig_name + '.eps', dpi=300) #save as EPS file (vector)
